In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

import itertools
import random
import os
import sys
import pickle

import src.pipe as pipe
import src.label_gen as label_gen
import src.entropy as entropy
import src.coverage as cov
import src.occurence_estimation as occ

In [4]:
from time import time
from tqdm import tqdm

### Load Test Datasets
- ACSIncome (data/df_ACSIncome.csv)
- IMDb Top 7000 Movies (data/imdb.csv)
- BlueNile (data/df_diamonds.csv)

In [5]:
acs_income = pd.read_csv("data/df_ACSIncome_enc_num.csv")

In [6]:
len(acs_income.value_counts())

200664

In [7]:
blue_nile = pd.read_csv("data/df_diamonds_enc_num.csv")

In [8]:
uk_roadsafety = pd.read_csv("data/df_uk_road_accident_enc_num.csv")

In [9]:
cov.combinatorial_sum(occ.get_cardinalities(uk_roadsafety))

84998143

### Perform Runtime Tests

In [10]:
def get_itemdict_of_combinatorial_sum(df):
    unique_values = {col: df[col].unique() for col in df.columns}
    cardinality = [f"{col}#{len(unique_values[col])}" for col in unique_values.keys()]
    card_ps = itertools.chain.from_iterable(
        itertools.combinations(cardinality, r) for r in range(2, len(cardinality) + 1)
    )

    cardinality_combinations = {
        tuple(col.split("#")[0] for col in i): cov.combinatorial_sum([int(col.split("#")[1]) for col in i])
        for i in tqdm(card_ps)
    }

    # Sorting the combinations by their combinatorial sum values
    cardinality_combinations = dict(sorted(cardinality_combinations.items(), key=lambda item: item[1]))

    selected_combinations = {}
    # sample the cardinality_combinations dictionary to get 12 combinations with an even distribution of the combinatory sum values
    for i in range(0, len(cardinality_combinations), int(len(cardinality_combinations)/12)):
        selected_combinations[list(cardinality_combinations.keys())[i]] = list(cardinality_combinations.values())[i:i+1][-1]
    
    #check if all combination length are represented else add the largest combination for that length
    for i in range(2, len(cardinality)+1):
        if i not in [len(key) for key in selected_combinations.keys()]:
            selected_combinations[list(cardinality_combinations.keys())[i-1]] = list(cardinality_combinations.values())[i-1:i][-1]

    #add the three last elements of the cardinality_combinations dictionary if combinatorial sum not already in selected_combinations
    for i in range(len(cardinality_combinations)-3, len(cardinality_combinations)):
        if list(cardinality_combinations.values())[i] not in selected_combinations.values():
            selected_combinations[list(cardinality_combinations.keys())[i]] = list(cardinality_combinations.values())[i:i+1][-1]

    #sort cardinality combinations by value
    selected_combinations = dict(sorted(selected_combinations.items(), key=lambda item: item[1]))

    return selected_combinations

In [11]:
def rowbased_runtime_performance(name, df, buckets, max_level=None):
    runtime_dict = {}
    output_dict = {}
    for cols, combination_size in buckets.items():
        t = time()
        freq_count_rows, keys_rows = occ.calc_occurences_countmin_rowbased_traversal(df[list(cols)], max_level=max_level, width=True)
        runtime = time() - t
        memory_size_keys = sum([sys.getsizeof(v) for v in keys_rows.values()])
        memory_size_cms = freq_count_rows.size()
        no_of_keys = sum(len(v) for v in keys_rows.values())
        runtime_dict[no_of_keys] = [runtime, combination_size, memory_size_keys, memory_size_cms]
        output_dict[no_of_keys] = [freq_count_rows, keys_rows]
    with open(f"results/{name}_rowbased_output.pkl", "wb") as f:
        pickle.dump(output_dict, f)
    del freq_count_rows, keys_rows
    return runtime_dict

In [12]:
def columnbased_runtime_performance(name, df, buckets, max_level=None):
    runtime_dict = {}
    output_dict = {}
    for cols, combination_size in buckets.items():
        t = time()
        freq_count_cols, keys_cols = occ.calc_occurences_countmin_columnbased_groupby(df[list(cols)], max_level=max_level, width=True)
        runtime = time() - t
        memory_size_keys = sum([sys.getsizeof(v) for v in keys_cols.values()])
        memory_size_cms = freq_count_cols.size()
        no_of_keys = sum(len(v) for v in keys_cols.values())
        runtime_dict[no_of_keys] = [runtime, combination_size, memory_size_keys, memory_size_cms]
        output_dict[no_of_keys] = [freq_count_cols, keys_cols]
    with open(f"results/{name}_columnbased_output.pkl", "wb") as f:
        pickle.dump(output_dict, f)
    del freq_count_cols, keys_cols
    return runtime_dict

In [13]:
def exact_runtime_performance(name, df, buckets, max_level=None):
    runtime_dict = {}
    output_dict = {}
    for cols, combination_size in buckets.items():
        t = time()
        freq_count_cols, keys_cols = occ.calculate_rowbased_exact(df[list(cols)], max_level=max_level)
        runtime = time() - t
        memory_size_keys = sum([sys.getsizeof(v) for v in keys_cols.values()])
        memory_size_dict = sys.getsizeof(freq_count_cols)
        no_of_keys = sum(len(v) for v in keys_cols.values())
        runtime_dict[no_of_keys] = [runtime, combination_size, memory_size_keys, memory_size_dict]
        output_dict[no_of_keys] = [freq_count_cols, keys_cols, ]
    with open(f"results/{name}_exact_output.pkl", "wb") as f:
        pickle.dump(output_dict, f)
    del freq_count_cols, keys_cols
    return runtime_dict

In [14]:
def calculate_runtime_values(name, df, buckets, max_level):
    runtime_dict_rows = {}
    runtime_dict_columns = {}
    # Group-By Rowbased CountMin Sketch [Proposed Approach]
    # Dependent on no. of unique rows in the dataset (base patterns)
    runtime_dict_rows = rowbased_runtime_performance(name, df, buckets, max_level=max_level)
    # Group-By Columnbased CountMin Sketch [Baseline Approach]
    # Dependent on no. of unique combinations in the dataset
    runtime_dict_columns = columnbased_runtime_performance(name, df, buckets, max_level=max_level)
    # runtime_dict_exact = exact_runtime_performance(name, df, buckets, max_level=max_level)
    runtime_dict_exact = {}
    return runtime_dict_rows, runtime_dict_columns, runtime_dict_exact

In [129]:
def visualize_runtime_experiment(name, runtime_dict_rows, runtime_dict_columns, runtime_dict_exact):   
    #visualize runtime_dict as line scatterplot for each bucket and logaritmic x-axis using runtime[0] and show runtime[1] as bars behind the lineplot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_rows.keys()], y=[i[0] for i in runtime_dict_rows.values()], mode='lines+markers', name='Runtime (Rowbased CMS)', line=dict(color='red')))
    #visualize runtime_dict_columns as line scatterplot
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_columns.keys()], y=[i[0] for i in runtime_dict_columns.values()], mode='lines+markers', name='Runtime (Columnbased CMS)', line=dict(color='grey')))
    #visualize runtime_dict_exact as line scatterplot
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_exact.keys()], y=[i for i in runtime_dict_exact.values()], mode='lines+markers', name='Runtime (DataCube)', line=dict(color='black')))
    # cut off y axis at max runtime of runtime_dict_columns or min of runtime_dict_exact, whichever is higher
    max_value = max([max([i[0] for i in runtime_dict_columns.values()]), min(runtime_dict_exact.values())])
    fig.update_yaxes(range=[0, 1.1*max_value])
    fig.update_layout(title=f'Runtime of Proposed Approach for {name}', xaxis_title='Number of Patterns', yaxis_title='Runtime (s)', xaxis_type="log")
    fig.show()

In [165]:
def get_pattern_length_viz_data_runtime(dict_output):
    runtime_output_uk_viz = {}
    for k, _v in dict_output.items():
        pattern_length = len(_v[-1])
        if pattern_length in runtime_output_uk_viz.keys():
            if _v[0] > runtime_output_uk_viz[pattern_length]:
                runtime_output_uk_viz[pattern_length] = _v[0]
        else:
            runtime_output_uk_viz.setdefault(pattern_length, _v[0])
    return runtime_output_uk_viz

In [181]:
def get_combination_length_viz_data_runtime(dict_output):
    runtime_output_uk_viz = {}
    for k, _v in dict_output.items():
        runtime_output_uk_viz.setdefault(k, _v[0])
    return runtime_output_uk_viz

In [189]:
def visualize_runtime_pattern_experiment(name, runtime_dict_rows, runtime_dict_columns, runtime_dict_exact):
    #visualize runtime_dict as line scatterplot for each bucket and logaritmic x-axis using runtime[0] and show runtime[1] as bars behind the lineplot
    fig = go.Figure()
    runtime_dict_rows = get_combination_length_viz_data_runtime(runtime_dict_rows)
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_rows.keys()], y=[i for i in runtime_dict_rows.values()], mode='lines+markers', name='Runtime (Rowbased CMS)', line=dict(color='red')))
    #visualize runtime_dict_columns as line scatterplot
    runtime_dict_columns = get_combination_length_viz_data_runtime(runtime_dict_columns)
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_columns.keys()], y=[i for i in runtime_dict_columns.values()], mode='lines+markers', name='Runtime (Columnbased CMS)', line=dict(color='grey')))
    #visualize runtime_dict_exact as line scatterplot
    runtime_dict_exact = get_combination_length_viz_data_runtime(runtime_dict_exact)
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_exact.keys()], y=[i for i in runtime_dict_exact.values()], mode='lines+markers', name='Runtime (DataCube)', line=dict(color='black')))
    # cut off y axis at max runtime of runtime_dict_columns or min of runtime_dict_exact, whichever is higher
    # max_value = max([max([i for i in runtime_dict_columns.values()]), min(runtime_dict_exact.values())])
    # fig.update_yaxes(range=[0, 1.1*max_value])
    fig.update_layout(title=f'Runtime of Proposed Approach for {name}', xaxis_title='Number of Patterns', yaxis_title='Runtime (s)', xaxis_type="log", yaxis_type="log")
    fig.show()

In [16]:
def visualize_memory_size_experiment(name, runtime_dict_rows, runtime_dict_columns, runtime_dict_exact):
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_rows.keys()], y=[i[3]/1000000 for i in runtime_dict_rows.values()], mode='lines+markers', name='Memory Size (Rowbased CMS)', line=dict(color='red')))
    fig.add_trace(go.Scatter(x=[i for i in runtime_dict_columns.keys()], y=[i[3]/1000000 for i in runtime_dict_columns.values()], mode='lines+markers', name='Memory Size (Columnbased CMS)', line=dict(color='grey')))
    # fig.add_trace(go.Scatter(x=[i for i in runtime_dict_exact.keys()], y=[i[3]/1000000 for i in runtime_dict_exact.values()], mode='lines+markers', name='Memory Size (Rowbased Dict)', line=dict(color='black')))
    fig.update_layout(title=f'Memory Size of Proposed Approach for {name}', xaxis_title='Combinatorial Space')
    fig.update_layout(yaxis=dict(title='Memory Size (MB)', overlaying='y', side='left'))
    fig.update_layout(xaxis_type="log")
    fig.show()

In [85]:
def visualize_no_of_combinations(name, buckets, runtime_dict_rows):
    fig = go.Figure()
    fig.add_trace(go.Bar(x=[str(len(i)) for i in buckets.keys()], y=[i[1] for i in runtime_dict_rows.values()], name='Number of Possible Patterns'))
    fig.add_trace(go.Bar(x=[str(len(i)) for i in buckets.keys()], y=[i for i in runtime_dict_rows.keys()], name='Number of Existing Patterns', marker_color='grey'))
    fig.update_layout(title=f'Number of Combinations for Proposed Approach for {name}', xaxis_title='Number of Keys')
    fig.update_layout(yaxis=dict(title='Number of Patterns', overlaying='y', side='left', type='log'))
    fig.update_layout(xaxis=dict(title='Number of Columns', overlaying='x', side='bottom'))
    fig.update_xaxes(type='category')
    fig.show()

#### Frequency Count

Check Runtime for different with growing combination size?!

#### Create DataCube results
##### Create SQL queries to execute in PostgreSQL

In [274]:
# Given data
data = pd.read_csv("results/data_cube_pattern_runtime_results.csv", dtype={'dataset': str, 'no_attr': int, 'cols': str}, delimiter=";")

In [275]:
#data to list of lists
data = data.values.tolist()

In [ ]:
# Function to generate SQL queries
def generate_sql_query(row):
    table_name, _, columns,_ = row
    columns = columns.strip("()").replace("'", "").split(', ')
    
    # Constructing the SELECT statement
    select_statement = '\",\n\t\"'.join(columns)
    
    # Constructing the GROUP BY statement
    group_by_statement = "GROUP BY\n\tCUBE(\"" + '\",\n\t\"'.join(columns) + "\")"
    
    # Constructing the SQL query
    sql_query = f'SELECT\n\t\"{select_statement}\",\n\tCOUNT(*)\nFROM \"{table_name}\"\n{group_by_statement}\n\n'
    
    return sql_query

# Generating SQL queries for each row in the data
for row in data:
    sql_query = generate_sql_query(row)
    print(sql_query)

Load DataCube results

In [ ]:
import ast
dc_results = pd.read_csv("results/data_cube_pattern_runtime_results.csv", dtype={'dataset': str, 'no_attr': int, 'cols': object}, delimiter=";", thousands=",", converters={'cols': ast.literal_eval})
#get ACSIncome results in dict with no_keys as key and runtime as value
acs_income_runtime_dict_dc = {row[1]: [row[3], row[2]] for row in dc_results.values if row[0] == "ACSIncome"}
#sort by key
acs_income_runtime_dict_dc = dict(sorted(acs_income_runtime_dict_dc.items()))
#get BlueNile results in dict with no_keys as key and runtime as value
blue_nile_runtime_dict_dc = {row[1]: [row[3], row[2]] for row in dc_results.values if row[0] == "BlueNile"}
#sort by key
blue_nile_runtime_dict_dc = dict(sorted(blue_nile_runtime_dict_dc.items()))
#get UKRoadSafety results in dict with no_keys as key and runtime as value
uk_roadsafety_runtime_dict_dc = {row[1]: [row[3], row[2]] for row in dc_results.values if row[0] == "UKRoadSafety"}
#sort by key
uk_roadsafety_runtime_dict_dc = dict(sorted(uk_roadsafety_runtime_dict_dc.items()))

In [281]:
for k, v in acs_income_runtime_dict_dc.items():
    print(f"({k}, {v[0]})")

(2, 1.8)
(3, 3.7)
(4, 9.6)
(5, 31.0)
(6, 144.0)
(7, 230.0)
(8, 560.0)
(9, 1155.0)


#### Combination Size

##### ACSIncome

In [56]:
acs_income_buckets = get_itemdict_of_combinatorial_sum(acs_income)

502it [00:00, 34449.29it/s]


In [83]:
acs_income_cardinality = [len(acs_income[col].unique()) for col in acs_income.columns]
print(acs_income_cardinality)

[3, 7, 16, 5, 2, 50, 9, 50, 3]


In [ ]:
acs_runtime_dict_rows, acs_runtime_dict_columns, acs_runtime_dict_exact = calculate_runtime_values("acs_income", acs_income, acs_income_buckets, max_level=None)

In [ ]:
visualize_runtime_experiment("ACSIncome", acs_runtime_dict_rows, acs_runtime_dict_columns, acs_runtime_dict_exact)

In [98]:
with open('results_server/ACSIncome_columnbased_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    acs_runtime_dict_columns_s = pickle.load(f)
with open('results_server/ACSIncome_rowbased_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    acs_runtime_dict_rows_s = pickle.load(f)
# with open('results_server/ACSIncome_exact_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    # acs_runtime_dict_exact_s ={}

In [190]:
visualize_runtime_pattern_experiment("ACSIncome", acs_runtime_dict_rows_s, acs_runtime_dict_columns_s, acs_income_runtime_dict_dc)

In [180]:
#show number of potential combinations for every number of keys with plotly
visualize_no_of_combinations("ACSIncome", acs_income_buckets, acs_runtime_dict_rows_s)

Check Memory Size for growing combination size?!

-  memory of CMS = ( width * depth * cell_size ) + HLL size
    - width is the width of the CMS and power of 2
    - depth is the depth of the CMS and defaults to 8
    - cell_size is the size of each cell in bytes and defaults to 4
    - HLL size is the size of the HyperLogLog used for cardinality estimation and defaults to 64KB

In [ ]:
#visualize memory size im MegaBytes as lineplot with markers 
visualize_memory_size_experiment("ACSIncome", acs_runtime_dict_rows, acs_runtime_dict_columns, acs_income_runtime_dict_exact)

##### BlueNile

Runtime Viz

In [21]:
bluenile_buckets = get_itemdict_of_combinatorial_sum(blue_nile)

120it [00:00, 54571.88it/s]


In [ ]:
bluenile_runtime_dict_rows, bluenile_runtime_dict_columns, bluenile_runtime_dict_exact = calculate_runtime_values("BlueNile", blue_nile, bluenile_buckets, max_level=None)

In [77]:
with open('results_server/BlueNile_columnbased_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    bluenile_runtime_dict_columns_s = pickle.load(f)
# with open('results_server/BlueNile_exact_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    bluenile_runtime_dict_exact_s = {}
with open('results_server/BlueNile_rowbased_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    bluenile_runtime_dict_rows_s = pickle.load(f)

In [150]:
visualize_runtime_experiment("Blue Nile", bluenile_runtime_dict_rows_s, bluenile_runtime_dict_columns_s, blue_nile_runtime_dict_dc)

Number of Patterns

In [ ]:
visualize_runtime_experiment("Blue Nile", bluenile_runtime_dict_rows, bluenile_runtime_dict_columns, bluenile_runtime_dict_exact)

In [149]:
visualize_no_of_combinations("BlueNile", bluenile_buckets, bluenile_runtime_dict_rows_s)

In [ ]:
visualize_memory_size_experiment("BlueNile", bluenile_runtime_dict_rows, bluenile_runtime_dict_columns, bluenile_runtime_dict_exact)

##### UK Road Safety Data

Runtime Viz

In [136]:
uk_roadsafety_buckets = get_itemdict_of_combinatorial_sum(uk_roadsafety)

502it [00:00, 36660.82it/s]


In [ ]:
uk_runtime_dict_rows, uk_runtime_dict_columns, uk_runtime_dict_exact = calculate_runtime_values("UK Road Safety", uk_roadsafety, uk_roadsafety_buckets, max_level=None)

In [141]:
with open('results_server/UKRoadSafety_columnbased_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    uk_runtime_dict_columns_s = pickle.load(f)
# with open('results_server/UKRoadSafety_exact_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    # uk_runtime_dict_exact_s = pickle.load(f)
with open('results_server/UKRoadSafety_rowbased_runtime_freq_output_2023-10-16.pkl', 'rb') as f:
    uk_runtime_dict_rows_s = pickle.load(f)

In [148]:
visualize_runtime_experiment("UK RoadSafety", uk_runtime_dict_rows_s, uk_runtime_dict_columns_s, uk_roadsafety_runtime_dict_dc)

#### No. of Attributes of Interest

UK Road Safety Data

In [234]:
with open("results_server/UKRoadSafety_rowbased_runtime_freq_output_pattern_2023-10-17.pkl", "rb") as f:
    uk_runtime_dict_rows_pattern = pickle.load(f)
with open("results_server/UKRoadSafety_columnbased_runtime_freq_output_pattern_2023-10-17.pkl", "rb") as f:
    uk_runtime_dict_columns_pattern = pickle.load(f)

In [235]:
def visualize_pattern_runtime(name, runtime_dict_rows, runtime_dict_columns):
    #visualize runtime_dict as line scatterplot for each bucket and logaritmic x-axis using runtime[0] and show runtime[1] as bars behind the lineplot
    fig = go.Figure()
    fig.add_trace(go.Scatter(x=[i[5] for i in runtime_dict_rows.values()], y=[i[0] for i in runtime_dict_rows.values()], mode='lines+markers', name='Runtime (Rowbased CMS)', line=dict(color='red')))
    #visualize runtime_dict_columns as line scatterplot
    fig.add_trace(go.Scatter(x=[i[5] for i in runtime_dict_columns.values()], y=[i[0] for i in runtime_dict_columns.values()], mode='lines+markers', name='Runtime (Columnbased CMS)', line=dict(color='grey')))
    #visualize runtime_dict_exact as line scatterplot
    # fig.add_trace(go.Scatter(x=[i for i in runtime_dict_exact.keys()], y=[i for i in runtime_dict_exact.values()], mode='lines+markers', name='Runtime (DataCube)', line=dict(color='black')))
    # cut off y axis at max runtime of runtime_dict_columns or min of runtime_dict_exact, whichever is higher
    # max_value = max([max([i for i in runtime_dict_columns.values()]), min(runtime_dict_exact.values())])
    # fig.update_yaxes(range=[0, 1.1*max_value])
    fig.update_layout(title=f'Runtime of Proposed Approach for {name}', xaxis_title='Number of Patterns', yaxis_title='Runtime (s)')
    fig.show()

In [236]:
visualize_pattern_runtime("UK RoadSafety", uk_runtime_dict_rows_pattern, uk_runtime_dict_columns_pattern)

In [259]:
dc_uk = pd.read_csv("results/data_cube_pattern_runtime_results.csv", dtype={'dataset': str, 'no_attr': int}, delimiter=";", thousands=",", converters={'cols': ast.literal_eval})
dc_uk = dc_uk[dc_uk['dataset'] == "UKRoadSafety"]
dc_uk['pattern_length'] = dc_uk['cols'].apply(lambda x: len(x))
dc_uk['pattern_length'] = dc_uk['pattern_length'].astype('category')
dc_uk_dict = dc_uk.groupby('pattern_length')['runtime'].max().to_dict()
for k,v in dc_uk_dict.items():
    print(f"({k}, {v})")

(2, 1.8)
(3, 4.1)
(4, 9.3)
(5, 26.0)
(6, 87.0)
(7, 240.0)
(8, 599.0)
(9, 1288.0)


In [243]:
for k, v in uk_runtime_dict_rows_pattern.items():
    print(f"({v[5]}, {v[0]})")

(2, 0.2613692283630371)
(3, 0.3493938446044922)
(4, 0.5035927295684814)
(5, 1.178410530090332)
(6, 3.792604684829712)
(7, 12.489306211471558)
(8, 37.236555099487305)
(9, 112.57445883750916)


In [244]:
for k, v in uk_runtime_dict_columns_pattern.items():
    print(f"({v[5]}, {v[0]})")

(2, 0.6560008525848389)
(3, 1.6113111972808838)
(4, 3.9000840187072754)
(5, 9.28112006187439)
(6, 21.775065422058105)
(7, 50.760629177093506)
(8, 115.7346031665802)
(9, 270.5235414505005)


In [247]:
for k, v in uk_runtime_dict_rows_pattern.items():
    print(f"({v[5]}, {k})")

(2, 147)
(3, 1061)
(4, 6005)
(5, 39712)
(6, 161973)
(7, 543677)
(8, 1383556)
(9, 3723646)


BlueNile Data

In [238]:
with open("results_server/BlueNile_rowbased_runtime_freq_output_pattern_2023-10-17.pkl", "rb") as f:
    bn_runtime_dict_rows_pattern = pickle.load(f)
with open("results_server/BlueNile_columnbased_runtime_freq_output_pattern_2023-10-17.pkl", "rb") as f:
    bn_runtime_dict_columns_pattern = pickle.load(f)

In [239]:
visualize_pattern_runtime("Blue Nile", bn_runtime_dict_rows_pattern, bn_runtime_dict_columns_pattern)

In [248]:
for k, v in bn_runtime_dict_rows_pattern.items():
    print(f"({v[5]}, {v[0]})")

(2, 0.017343521118164062)
(3, 0.021414995193481445)
(4, 0.06624794006347656)
(5, 0.1814861297607422)
(6, 0.5705170631408691)
(7, 1.9160737991333008)


In [250]:
for k, v in bn_runtime_dict_columns_pattern.items():
    print(f"({v[5]}, {v[0]})")

(2, 0.031088590621948242)
(3, 0.0706183910369873)
(4, 0.17522835731506348)
(5, 0.41147828102111816)
(6, 0.9748408794403076)
(7, 2.3847076892852783)


In [251]:
for k, v in bn_runtime_dict_rows_pattern.items():
    print(f"({v[5]}, {k})")

(2, 112)
(3, 792)
(4, 4817)
(5, 14535)
(6, 43089)
(7, 129194)


ACSIncome

In [240]:
with open("results_server/ACSIncome_rowbased_runtime_freq_output_pattern_2023-10-17.pkl", "rb") as f:
    acs_runtime_dict_rows_pattern = pickle.load(f)
with open("results_server/ACSIncome_columnbased_runtime_freq_output_pattern_2023-10-17.pkl", "rb") as f:
    acs_runtime_dict_columns_pattern = pickle.load(f)

In [241]:
visualize_pattern_runtime("ACSIncome", acs_runtime_dict_rows_pattern, acs_runtime_dict_columns_pattern)

In [252]:
for k, v in acs_runtime_dict_rows_pattern.items():
    print(f"({v[5]}, {v[0]})")

(2, 0.234544038772583)
(3, 0.44391298294067383)
(4, 0.8890981674194336)
(5, 2.536705732345581)
(6, 8.241523027420044)
(7, 24.016265869140625)
(8, 70.01660227775574)
(9, 196.24791169166565)


In [253]:
for k, v in acs_runtime_dict_columns_pattern.items():
    print(f"({v[5]}, {v[0]})")

(2, 0.5493929386138916)
(3, 1.3513646125793457)
(4, 3.1251931190490723)
(5, 7.724844694137573)
(6, 20.108131885528564)
(7, 48.380892276763916)
(8, 123.26879978179932)
(9, 358.2201929092407)


In [254]:
for k, v in acs_runtime_dict_rows_pattern.items():
    print(f"({v[5]}, {k})")

(2, 2182)
(3, 16049)
(4, 51150)
(5, 185276)
(6, 576907)
(7, 1490915)
(8, 3830436)
(9, 9339813)


### Examine CMS Approach


In [ ]:
d = acs_income[["gender", "education", "income", "race", "age", "workclass"]]

In [ ]:
w_ = len(d.value_counts().to_dict()) * len(list(occ.powerset(d.columns))) / 10
2**int(np.log2(w_))

In [ ]:
w_ = cov.combinatorial_sum([len(d[col].unique()) for col in d.columns]) / 10
w_ = 2**int(np.log2(w_))
w_

In [ ]:
w_ = len(d.drop_duplicates()) * len(list(occ.powerset(d.columns))) / 10


In [ ]:
w__ = len(d.value_counts()) * len(list(occ.powerset(d.columns))) / 10
2**int(np.log2(w__))

In [ ]:
m = (w_ * 12 * 4) + 64000
m // (1024*1024)

In [ ]:
#one order of magnitude of max. expected number of keys
w = len(d.value_counts().to_dict()) * len(list(occ.powerset(d.columns)))/ 10
#closest number that is power of 2 to w
w = 2**int(np.log2(w))
if w < 32768:
    w = 32768
if w > 8388608:
    w = 8388608

In [ ]:
test_freq, test_keys = occ.calc_occurences_countmin_rowbased_traversal(d, max_level=None, width=True)

In [ ]:
test_freq.size() // (1024*1024)

In [ ]:
test_freq.quality()

In [ ]:
base_dict = acs_income.value_counts().to_dict()
key_dict = {}
for k, v in tqdm(base_dict.items()):
    # create dict of all combinations of k as keys and v as value
    p_i = occ.powerset(k)
    for j in p_i:
        j = str(j)
        key_dict.setdefault(j, 0)
        key_dict[j] += v

In [ ]:
from bounter import CountMinSketch
groupby_col_combs = CountMinSketch(128)

In [ ]:
groupby_col_combs.update(key_dict)

In [ ]:
groupby_col_combs.cardinality()

In [ ]:
groupby_col_combs.quality()

In [ ]:
len(key_dict)

In [ ]:
groupby_col_combs.size() // (1024*1024)

In [ ]:
#get size in MB of key_dict
sum([sys.getsizeof(v) for v in key_dict.values()]) / (1024*1024)

In [ ]:
differences = {}
for k,v in key_dict.items():
    if v != groupby_col_combs[k]:
        differences[k] = [v, groupby_col_combs[k]]

pd.DataFrame(differences).T

In [ ]:
# error = 2 * n / w
n = len(key_dict)
error = 0.1
w = 2 * n / error
w = 2**int(np.log2(w))
w

In [ ]:
len(key_dict)

In [ ]:
((w * 8 * 4) + 64000) / (1024*1024)